# Clustering

Si vous vous rappelez bien du chapitre "Eléments de définition", il y a trois types principaux de Machine Learning; Supervisé, non supervisé et par renforcement. Jusqu'à maintenant nous n'avions que des cas d'apprentissage supervisé (nous avions les labels), il est donc temps de parler de l'apprentissage non-supervisé (nous n'avons pas les labels). 

Imaginons que nous avons des images de chiens et de chats qui ne sont pas étiquettées. Une solution serait de payer des gens pour ajouter les labels, mais elle est couteuse en temps et en argent. Une autre est d'utiliser du clustering. Cette approche va essayer de créer des groupes (clusters) en utilisant les données. 

Une idée serait de partir du principe que les données de même classe (chiens et chats) sont proches dans l'espace définit par les pixels de l'image. Pour les regrouper, nous pourrions avoir deux points, un pour les chiens et un pour les chats. Nous classifierons l'image en fonction du point duquel elle est le plus proche. La question est donc: Comment trouver ces points? 

La réponse est simple, en utilsant Kmeans!

## Kmeans

L'algorithm se base sur ce qu'on appel des centroids, ce sont des points qui seront placé au milieu des clusters. Chaque échantillon fait partie de la catégorie dont le centroid est le plus proche. La procédure se passe en deux parties
1. Attribution des échantillons aux clusters
2. Mise à jour des centroids

### En détail
Afin de nous faciliter la tâche, nous allons introduire une matrice binaire $z$ qui est de taille $N\times K$, $N$ détermine le nombre de points de données et $K$ le nombre de clusters que nous cherchons. De cette façon, l'entrée $z_{1,1}=0$ voudra dire que $x_1$ n'est pas dans le cluster 1, et $z_{1,2}=1$ représente l'appartenance du point $x_1$ au cluster 2. Nous avons donc la contrainte qu'il ne peut y avoir que une entrée qui vaut 1 et le reste doit être 0, mathématiquement, cela donne $\sum_k z_{n,k} = 1$. 
<br> Maintenant que cela est clair, nous pouvons (enfin) décrire l'algorithm.

### L'algorithme
1. On initialise les centroids $\mu_n$. Pour l'instant nous le faisons au hasard. 
2. On attribue le cluster de chaque données $x_n$ au centroid le plus proche et remplissons la matrice $z$.<br> Comme avec Knn. nous pouvons prendre la mesure de distance qui nous convient le mieux. Dans notre cas nous utiliserons l'euclidienne. 
3. On met les centroids à jour en calculant la moyenne des points présent dans le cluster
$$\mu_k = \frac{\sum_{n=1}^N z_{nk}\textbf{x}_n}{\sum_{n=1}^N z_{nk}}$$
4. On retourne au point 2, jusqu'à ce que $z$ ne change plus. 

Ce problème peut-être décrit par une fonction de couts que nous voulons minimiser:
			
$$min\sum_n\sum_k z_{n,k} (x_n-\mu_k)^T(x_n-\mu_k)$$
$$\textbf{s.t.: } z_{n,k}\in \{0,1\} \text{ et } \sum_k z_{n,k} = 1 $$

Le but est de rendre la somme des distances la plus petite possible. Kmeans va converger à chaque fois mais vu que les centroids sont initilisés aléatoirement et que le problême n'est pas convex (une seule solution) chaque fois que nous lançons Kmeans nous aurons potentiellement une solution différente.

### Soucis et Solutions
Il y a deux problèmes majeurs, comment déterminer le nombre de clusters. Dans certains cas nous avons énorméments de données sans labels et il est impossible de déterminer le nombre de cluster présent. Le deuxième est l'initialisation des centroids. Vu que l'initilisation a un impact majeur sur la solution finale, il faut le faire de manière intelligent sans risquer de trop biaiser le résultat. 

#### Initlisation des centroids
Il existe plusieurs approches plus ou moins efficaces comme l'initilisation aléatoire dans une certaine "zone", prendre $K$ points dans les données et utiliser leur position comme centroid. Dans les deux cas, nous avons potentiellement des centroids très proche l'un de l'autre, ce qui ralentit la convergence. L'approche la plus efficace (à ce jour) se nomme kmeans++ et est celle utilisée par la libraire Scikit-Learn. 

L'idée est assez simple, nous choisissons un point dans $X$ aléatoirement et utilisons ses coordonnées comme coordonnées pour $\mu_1$. Ensuite nous trouvons le point le plus éloigné de $\mu_1$ et l'utilisons pour déterminer $\mu_2$. Puis, nous trouvons le points le plus éloigné de $\mu_1$ et $\mu_2$, etc. 

Cette approche nous donne une approximation $O(logK)$ de la réponse finale. 

#### Choix du nombre de cluster
Il existe plusieurs approches, celle décrite ici est basé sur ce qu'on appele la variance intra-cluster. La variance d'un cluster est déterminé par:
$$W_k:=\frac{1}{|\mu_k|}\sum_{x\in \mu_k}(x-\mu_k)^2$$
La variance total est la somme de tous les $W_k$. 

Le but est de choisir le nombre de cluster $K$ qui minimise la variance intra-cluster.

## À savoir
Étant donnée que nous n'avons pas les labels, il n'est, la plupart du temps, pas possible d'évaluer un algorithm de clustering. Il faudra soit lui faire confiance ou regarder par soit même. De plus, on sait que Kmeans va toujours converger, il va donc toujours donner une infomation. Dans le cas des images de chien et de chat, il trouvera autant de cluster qu'il y a d'images si on le lui demande. Il faut donc être prudent lors de l'utilisation de l'algorithm. 

### À retenir

- d

## Implémentation

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [4]:
'''
-- Création de l'ensemble de donnée --
Notez que j'utilise une distribution normal ce qui permet d'avoir tous les points d'une même class relativement proche.
De plus, je génère un ensemble balancé
'''

elements = 50
random_state = 42
K = 5

def generate_data(elements, random_state):
    rng=np.random.RandomState(random_state)

    classes_x = rng.normal(0, 0.55, (2,elements))
    classes_y = rng.normal(1, 0.55, (2,elements))
    labels = np.concatenate((np.zeros(elements), np.ones(elements)))
    dset = np.zeros((2,elements*2))

    dset[0] = np.concatenate((classes_x[0], classes_y[0]))
    dset[1] = np.concatenate((classes_x[1], classes_y[1]))

    return dset, labels

class Kmeans:
    def __init__(self, K):
        self.K = K
        self.centroids = None

    def fit(self, data):
        self.centroids = np.random.uniform(np.min(data), np.max(data), size=(self.K, len(data[0])))
        #print(self.centroids)

        prev_z = None
        curr_z = None
      
        while type(prev_z) == type(None) or (prev_z != curr_z).any():
            prev_z = curr_z
            curr_z = np.zeros((len(data), self.K))
            #print(f"Prev_z: {prev_z}")
            #print(f"Curr_z: {curr_z}")

            for ind_point, dpoint in enumerate(data):
                dist = np.zeros(self.K)

                for ind_centr, centroid in enumerate(self.centroids):
                    dist[ind_centr] = self.distance(dpoint, centroid)                
                curr_z[ind_point][np.argmin(dist)] = 1
            
            for ind_centr, centroid in enumerate(self.centroids): 
                new_centr = np.zeros(len(centroid))

                for dim in range(len(centroid)):
                    new_centr[dim] = np.dot(curr_z[:, ind_centr],data[:,dim])/sum(curr_z[:, ind_centr])

                self.centroids[ind_centr] = new_centr
            
            print(self.centroids)

    def kmeans_pp(self, data):
        pass
    

    def predict(self, data_point):
        preds = []

        for dp in data_point:
            distances = []

            for point in self.centroids:
                distances.append(self.distance(dp, point))

            #sorted_distances = np.argsort(distances)
            preds.append(np.argmin(distances))
        
        return preds
        

    def distance(self, x_new, x):
        return np.power(np.dot(x_new-x, x_new-x), 2)

data, label = generate_data(elements, random_state)

data[0,:] = (data[0,:]-min(data[0,:]))/(max(data[0,:])-min(data[0,:]))
data[1,:] = (data[1,:]-min(data[1,:]))/(max(data[1,:])-min(data[1,:]))

# print(data)

kmeans = Kmeans(2)

kmeans.fit(data.T)

print(f"Final:\n {kmeans.centroids}")

preds = []
[preds.append(str(pred)) for pred in kmeans.predict(data.T)]

'''
-- Plot --
'''

fig0 = px.scatter(x=data[0,:], y=data[1,:], color=preds, color_discrete_sequence=["rgb(2,48,71)", "rgb(255,158,2)"])
fig1 = px.scatter(x=kmeans.centroids.T[0,:], y=kmeans.centroids.T[1,:], color=["Centroid 0","Centroid 1"], symbol_sequence=["square"], color_discrete_sequence=["rgb(255,158,2)", "rgb(2,48,71)"])

fig1.update_traces(marker=dict(size=10,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig2 = go.Figure(data=fig0.data + fig1.data)
fig2.show('iframe')

[[0.5555255  0.57978013]
 [0.20073797 0.3382875 ]]
[[0.58811734 0.60681782]
 [0.24004356 0.35860791]]
[[0.60088987 0.62040944]
 [0.25526379 0.36440545]]
[[0.61470668 0.63285889]
 [0.26909674 0.37252295]]
[[0.61470668 0.63285889]
 [0.26909674 0.37252295]]
Final:
 [[0.61470668 0.63285889]
 [0.26909674 0.37252295]]
